<a href="https://colab.research.google.com/github/jasonwong-lab/HKU-Practical-Bioinformatics/blob/main/NGS_sequence_alignment_command_line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BBMS3004 - Targeted sequencing in acute myeloid leukaemia

*6 February 2026*

*Alvin Ip*

This practical aims to demonstrate how next-generation sequencing data is processed to yield clinically useful information. It is a highly condensed version of BIOF3002 Clinical Genomics and Bioinformatics that Prof Jason Wong and I co-coordinated for bioinformatics students.

In this session. we will build a toy bioinformatic pipeline to get a feel of what it is like to practice bioinformatics using Linux. Our platform will be [Google Colab](https://colab.research.google.com/), which is a free cloud computing platform. It uses Jupyter notebook, a convenient platform for programming in Python. As we will be using Linux commands in our session, we will exploit the "!" command in Python function to turn our script into system command expressed in Linux. Don't worry. This session does not assume any programming skills from you. The Jupyter notebook setup will take you through the analysis. It would be very beneficial if you can slowly read through the comments (i.e. those lines starting with "#") and the commands (i.e. the lines that do not start with "#") before you run them and try to understand what they are doing.

### GATK Best Practice for Somatic SNVs and Indels

We will try to build a pipeline according [GATK best practices for data preprocessing](https://gatk.broadinstitute.org/hc/en-us/articles/360035535912) and [GATK best practices for somatic SNVs and indels](https://gatk.broadinstitute.org/hc/en-us/articles/360035894731-Somatic-short-variant-discovery-SNVs-Indels-). Other GATK Best Practices pipelines can be found in the menu of the website as well.

### Building the Toy Pipeline

While we are trying to build a functional bioinformatic pipeline, but as some elements of this pipeline has been abridged to suit the computer resources and running time in the teaching session, this pipeline should NOT be used for any other purposes, including clinical practice and research.

### Package installation and downloads for workshop (~ 5-10 minutes)

1.   conda (for simple installation of packages)
2.   FastQC (for reads quality check)
3.   Trimmomatic (for trimming low-quality bases and adapters)
4.   bwa (tools for sequence alignment)
5.   samtools (tools for processing sam & bam files)
6.   IGV (for visualising sequencing data and variants)  

**IMPORTANT：Every time you connect to Google Colab, you have to perform these set up steps again.**

In [ ]:
# Install terminal for Google Colab (optional)
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm

In [ ]:
# Set working pathway to your own Google drive, so that the data files are saved after closing Colab (~ 1 min)
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Install igv-notebook (<1 min)
!pip install igv-notebook

In [ ]:
# Install conda (~ 1 min). There will be a message saying that the session has crashed, but don't worry about this. This is due to the session restarting following conda installation.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Install fastqc (~ 2 mins)
!conda install -c bioconda fastqc

In [ ]:
# Install bwa (~ 1 min)
!conda install -c bioconda bwa

In [ ]:
# Install samtools, bcftools, htslib, bedtools, bamtools (~ 2 mins)
!conda install -y -c conda-forge -c bioconda \
  samtools=1.20 bcftools=1.20 htslib=1.20 bedtools=2.31.1 bamtools

In [ ]:
# Check installation ran correctly
!samtools --version | head -1

In [ ]:
# Install GATK in a new environment (to specify the exapct openjdk and python version) ~1.5 mins
!conda create -y -n gatkenv python=3.10
!conda config --env --set channel_priority strict
!conda install -y -n gatkenv -c conda-forge -c bioconda gatk4=4.6.2.0 openjdk=17

In [ ]:
# Check that gatk is installed properly
!conda run -n gatkenv gatk --version

### Set working directory

By default, the working directory will be My Drive/bbms3004

In [ ]:
import os
try:
  os.mkdir("/content/gdrive/My Drive/bbms3004")         # change this path if necessary
except FileExistsError:
  print("directory already exist. OK to continue")
os.chdir("/content/gdrive/My Drive/bbms3004")

### Download the full toolkit that includes reference files, some tools and sequencing data for analysis.

In [ ]:
# Download data files from Google Drive if not already present (~ 3 min)
# Double check that we are in the right directory (~ 30s)
import os
os.chdir("/content/gdrive/MyDrive/bbms3004")                     # change this path if necessary

import os
if os.path.isfile("/content/gdrive/MyDrive/bbms3004/bbms3004_toolkit/17H0510082_1.fastq.gz"):    # check if the file exist
  print("Data files already exist, OK to continue.")
else:
  !pip install gdown
  !gdown -O bbms3004_toolkit.zip https://drive.google.com/uc?id=1TzOHbEKF2P1EWoxVsS_pvzS4ZS46XNXh # download if file not exist

  # unzip toolkit files
  !unzip -o bbms3004_toolkit.zip

  # remove the downloaded zip file after extraction to save space
  !rm bbms3004_toolkit.zip

# Check that the data files are present
!ls -l ./bbms3004_toolkit/

## Start processing the NGS sequencing data following these steps:

1. Quality control

2. Trimming of adapters and low-quality bases

3. Sequence alignment using Burrows–Wheeler Aligner (BWA)

4. Viewing SAM files

5. Mark and remove PCR duplicates

6. Base recalibration

7. Variant calling by Mutect2

8. Variant filtering by bcftools

9. Variant annotation by ANNOVAR

10. Manual curation of variants

11. Visualisation of curated variants in IGV



In [ ]:
# Check what datasets we have downloaded
%cd /content/gdrive/MyDrive/bbms3004/bbms3004_toolkit
!ls -l

%cd data
!ls -l

In [ ]:
# To look at the ChIP-seq fastq file type (head -n 12 to print first 12 lines):
!zcat 17H0510082_1.fastq.gz | head -n 12

In [ ]:
# How many reads are there in the file? Type your code below:


### 1. Quality control

In [ ]:
## Run FastQC
!fastqc 17H0510082_1.fastq.gz
!fastqc 17H0510082_2.fastq.gz

In [ ]:
# Check the html file output by FastQC
!ls

# Download the html file and check it on your local browser
from google.colab import files
files.download('17H0510082_1_fastqc.html')

You can also download the file **17H0510082_1_fastqc.html** from "gdrive/MyDrive/bbms3004/Datasets" on the left side.

### 2. Trimming of adapters and low-quality bases

In [ ]:
# Perform Trimmomatic
!java -jar ../Trimmomatic-0.39/trimmomatic-0.39.jar PE -phred33 \
    17H0510082_1.fastq.gz 17H0510082_2.fastq.gz \
    17H0510082_1_trimmed_paired.fq.gz 17H0510082_1_trimmed_unpaired.fq.gz 17H0510082_2_trimmed_paired.fq.gz 17H0510082_2_trimmed_unpaired.fq.gz \
    ILLUMINACLIP:../Trimmomatic-0.39/adapters/TruSeq3-PE-2.fa:2:30:10 LEADING:10 TRAILING:10 SLIDINGWINDOW:4:15 MINLEN:40


### 3. Sequence alignment using Burrows–Wheeler Aligner (BWA)

In [ ]:
# Look at the options for bwa and bwa mem
!bwa

In [ ]:
!bwa mem

In [ ]:
# Let's take a look at the database files
%cd /content/gdrive/My Drive/bbms3004/bbms3004_toolkit/Ref

!ls -l

In [ ]:
# Get ready to run BWA: First go into the Datasets directory
%cd /content/gdrive/My Drive/bbms3004/bbms3004_toolkit/data

In [ ]:
# Do sequence alignment with the default options
!bwa mem -t 4 -M ../Ref/ucsc.hg19.subset.fasta 17H0510082_1_trimmed_paired.fq.gz 17H0510082_2_trimmed_paired.fq.gz > 17H0510082.sam

### 4. Viewing SAM files

In [ ]:
# Check the result
!head -n 20 17H0510082.sam

In [ ]:
# Sort the SAM file and compress SAM to BAM
!conda run -n gatkenv gatk SortSam \
    -I 17H0510082.sam \
    -O 17H0510082_sorted.bam \
    --SORT_ORDER coordinate

In [ ]:
# Add read groups (RG), tags for downstream tools to run properly
!conda run -n gatkenv gatk AddOrReplaceReadGroups \
    -I 17H0510082_sorted.bam \
    -O 17H0510082_RG.bam \
    --RGID SPACE --RGLB panel --RGPL ILLUMINA --RGPU unit1 --RGSM 17H0510082 --CREATE_INDEX true

### 5. Mark and remove PCR duplicates

In [ ]:
# Perform MarkDuplicates to mark duplicate reads
!conda run -n gatkenv gatk MarkDuplicates \
    -I 17H0510082_RG.bam \
    -O 17H0510082_MD.bam \
    --METRICS_FILE 17H0510082_MD.stats \
    --CREATE_INDEX true

In [ ]:
# Inspect the duplicate rate of the sequencing data
# Try to locate the column with header "PERCENT_DUPLICATION"
!cat 17H0510082_MD.stats | sed -n '7,8p' | column -t -s $'\t'   # Extract the 7th and 8th lines and format as table

### 6. Base recalibration

In [ ]:
# Perform BaseRecalibrator to recalibrate base quality scores
!conda run -n gatkenv gatk BaseRecalibrator \
    -I 17H0510082_MD.bam \
    -R ../Ref/ucsc.hg19.subset.fasta \
    --known-sites ../Ref/dbsnp_138.hg19.vcf \
    --known-sites ../Ref/Mills_and_1000G_gold_standard.indels.hg19.vcf \
    -L ../Ref/myeloid-targets.interval_list \
    -ip 50 \
    -O 17H0510082_recal_data.table

# Apply the recalibration to the sequence data
!conda run -n gatkenv gatk ApplyBQSR \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -I 17H0510082_MD.bam \
    --bqsr-recal-file 17H0510082_recal_data.table \
    -O 17H0510082_BR.bam

#### Collect quality metrics before variant calling

In [ ]:
# Collect quality control metrics using the variant-call ready BAM file (~ 3 minutes)
# Collect QC metrics
!conda run -n gatkenv gatk CollectMultipleMetrics \
    -I 17H0510082_BR.bam \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -O 17H0510082_QC_metrics

# Collect read counts
!conda run -n gatkenv gatk CollectReadCounts \
    -I 17H0510082_BR.bam \
    -L ../Ref/myeloid-targets.interval_list \
    --interval-merging-rule OVERLAPPING_ONLY \
    --format TSV \
    -O 17H0510082_read_counts.table

# Collect HS metrics
!conda run -n gatkenv gatk CollectHsMetrics \
    -I 17H0510082_BR.bam \
    -O 17H0510082_hs_metrics.txt \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -BAIT_INTERVALS ../Ref/myeloid-probe-coords.interval_list \
    -TARGET_INTERVALS ../Ref/myeloid-targets.interval_list

In [ ]:
# Inspect the output from HS metrics
!cat 17H0510082_hs_metrics.txt | sed -n '7,8p' | awk '{print $13 "\t" $23 "\t" $34 "\t" $45 "\t" $48 "\t" $52 "\t" $53 "\t" $55 "\t" $56 "\t" $58}' | column -t -s $'\t'

### 7. Variant calling by Mutect2

In [ ]:
# Perform variant calling with Mutect2 (~ 5 minutes)
!conda run -n gatkenv gatk Mutect2 \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -I 17H0510082_BR.bam \
    -L ../Ref/myeloid-targets.interval_list \
    --germline-resource ../Ref/af-only-gnomad.myeloid.bedtools.vcf.gz \
    --f1r2-tar-gz f1r2.tar.gz \
    -O 17H0510082_unfiltered.vcf.gz

!conda run -n gatkenv gatk LearnReadOrientationModel \
    -I f1r2.tar.gz \
    -O read-orientation-model.tar.gz

!conda run -n gatkenv gatk GetPileupSummaries \
    -I 17H0510082_BR.bam \
    -V ../Ref/small_exac_common_myeloid.vcf.gz \
    -L ../Ref/small_exac_common_myeloid.vcf.gz  \
    -O getpileupsummaries.table

!conda run -n gatkenv gatk CalculateContamination \
    -I getpileupsummaries.table \
    -O contamination.table

!conda run -n gatkenv gatk FilterMutectCalls \
    -V 17H0510082_unfiltered.vcf.gz \
    -R ../Ref/ucsc.hg19.subset.fasta \
    --contamination-table contamination.table \
    --ob-priors read-orientation-model.tar.gz \
    -O 17H0510082_filtered.vcf.gz

In [ ]:
# Inspect the VCF file from Mutect2
!zcat 17H0510082_filtered.vcf.gz

### 8. Variant filtering by bcftools

In [ ]:
# Use bcftools to filter variants with variant allele frequencies less than 0.05
!bcftools filter -i " FORMAT/AF > 0.05 " 17H0510082_filtered.vcf.gz -o 17H0510082_filtered_AF05.vcf

### 9. Variant annotation by ANNOVAR

In [ ]:
# Perform variant annotation by ANNOVAR
!chmod -R 700 ../annovar   # give execute permission to annovar scripts
!perl ../annovar/table_annovar.pl 17H0510082_filtered_AF05.vcf \
    ../annovar/humandb/ \
    -buildver hg19 \
    -out 17H0510082_filtered_annotate \
    -remove \
    -protocol refGene,cosmic86,clinvar_20170905,exac03nontcga,gnomad_exome \
    -operation g,f,f,f,f \
    -nastring . \
    -vcfinput

### 10. Manual curation of variants

In [ ]:
# Download ANNOVAR annotated file for viewing and curation
from google.colab import files
files.download('17H0510082_filtered_annotate.hg19_multianno.txt')

### 11. Visualisation of curated variants in IGV

In [ ]:
## IGV
# Load track from local paths
import igv_notebook

igv_notebook.init()

# FLT3
b = igv_notebook.Browser(
    {
        "genome": "hg19",
        "locus": "chr13:28592610-28592670"
    }
)

b.load_track(
    {
        "name": "17H0510082",
        "path": "17H0510082_BR.bam",
        "indexPath": "17H0510082_BR.bai",
        "format": "bam",
        "type": "alignment"
    })

# NPM1
c = igv_notebook.Browser(
    {
        "genome": "hg19",
        "locus": "chr5:170837516-170837576"
    }
)

c.load_track(
    {
        "name": "17H0510082",
        "path": "17H0510082_BR.bam",
        "indexPath": "17H0510082_BR.bai",
        "format": "bam",
        "type": "alignment"
    })


### After the session

Thank you for going through this session. If you have any inquiry regarding the course content or issues with running the server, please do not hesitate to email me at iphowan@hku.hk